## Visualization scripts for single-player Type Through the Bible files

(Still very much a work in progress--many more analyses to come!)

By Ken Burchfiel

Released under the MIT License

In [1]:
import time
start_time = time.time()
import pandas as pd # Polars could also be used in place of Pandas to 
# speed up some import/analysis processes--but, if there ends up being
# a need for faster data transformation code, I would rather do it in 
# C++ (as a means of building up my practice with that language.)
pd.set_option('display.max_columns', 1000)
import plotly.express as px
import numpy as np

## Analyzing test result file:

In [2]:
df_tr = pd.read_csv('../Files/test_results.csv') # tr = 'test results'

#Converting start/end timestamps to DateTime values:
for col in ['Local_Test_Start_Time', 'Local_Test_End_Time']:
    df_tr[col] = pd.to_datetime(df_tr[col])
# Ensuring the tests are being displayed in chronological order:
# (This will help ensure that the 'Test_ID' values that we're about
# to create are accurate)
df_tr = df_tr.sort_values(
    'Local_Test_Start_Time').reset_index(drop=True).copy()

df_tr['Test number'] = df_tr.index + 1


df_tr

,Unix_Test_Start_Time,Local_Test_Start_Time,Unix_Test_End_Time,Local_Test_End_Time,Verse_ID,Verse_Code,Verse,Characters,WPM,Test_Seconds,Error_Rate,Error_and_Backspace_Rate,Marathon_Mode,Player,Tag_1,Tag_2,Tag_3,Test number
0,1750909743,2025-06-25 23:49:03-04:00,1750909747,2025-06-25 23:49:07-04:00,1,Genesis_1:1,"In the beginning, God created heaven and earth.",47,124.928204,4.514593,0.021277,0.042553,0,KJB3,Cherry_Red,NaN,NaN,1
1,1750910427,2025-06-26 00:00:27-04:00,1750910430,2025-06-26 00:00:30-04:00,1,Genesis_1:1,"In the beginning, God created heaven and earth.",47,181.050428,3.115154,0.000000,0.000000,0,KJB3,Cherry_Red,NaN,NaN,2
2,1750910839,2025-06-26 00:07:19-04:00,1750910854,2025-06-26 00:07:34-04:00,2,Genesis_1:2,"But the earth was empty and unoccupied, and da...",141,113.465015,14.912085,0.120567,0.205674,1,KJB3,Cherry_Red,NaN,NaN,3
3,1750910854,2025-06-26 00:07:34-04:00,1750910860,2025-06-26 00:07:40-04:00,3,Genesis_1:3,"And God said, ""Let there be light."" And light ...",53,110.773001,5.741471,0.018868,0.037736,1,KJB3,Cherry_Red,NaN,NaN,4
4,1750910860,2025-06-26 00:07:40-04:00,1750910869,2025-06-26 00:07:49-04:00,4,Genesis_1:4,"And God saw the light, that it was good; and s...",89,116.464066,9.170210,0.078652,0.157303,1,KJB3,Cherry_Red,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1752552952,2025-07-15 00:15:52-04:00,1752552962,2025-07-15 00:16:02-04:00,83,Genesis_4:3,"Then it happened, after many days, that Cain o...",101,124.332020,9.748092,0.059406,0.079208,1,KJB3,Cherry_Red,NaN,NaN,97
97,1752552962,2025-07-15 00:16:02-04:00,1752552975,2025-07-15 00:16:15-04:00,84,Genesis_4:4,Abel likewise offered from the firstborn of hi...,128,120.063965,12.793181,0.054688,0.109375,1,KJB3,Cherry_Red,NaN,NaN,98
98,1752552984,2025-07-15 00:16:24-04:00,1752552996,2025-07-15 00:16:36-04:00,85,Genesis_4:5,"Yet in truth, he did not look with favor on Ca...",120,121.676182,11.834691,0.058333,0.116667,0,KJB3,Cherry_Red,NaN,NaN,99
99,1752553003,2025-07-15 00:16:43-04:00,1752553009,2025-07-15 00:16:49-04:00,86,Genesis_4:6,"And the Lord said to him: ""Why are you angry? ...",74,143.780078,6.176099,0.000000,0.000000,0,KJB3,Cherry_Red,NaN,NaN,100


Calculating various timing statistics that will prove useful for endurance-related analyses:

In [3]:
for time_type in ['Start', 'End']:
    df_tr[f'{time_type} Date'] = df_tr[
        f'Local_Test_{time_type}_Time'].dt.date
    df_tr[f'{time_type} Hour'] = df_tr[
        f'Local_Test_{time_type}_Time'].dt.hour
    df_tr[f'{time_type} Minute'] = df_tr[
        f'Local_Test_{time_type}_Time'].dt.minute
    df_tr[f'{time_type} 30-Minute Block'] = np.where(
        df_tr[f'{time_type} Minute'] >= 30, 2, 1)
    # Using floor division to determine the 15- and 10-minute blocks
    # into which each test falls:
    df_tr[f'{time_type} 15-Minute Block'] = df_tr[
        f'{time_type} Minute'] // 15 + 1
    df_tr[f'{time_type} 10-Minute Block'] = df_tr[
        f'{time_type} Minute'] // 10 + 1

# Creating columns that will store unique starting hours and
# 30/15/10-minute blocks:
df_tr['Unique Hour'] = df_tr['Start Date'].astype(
    'str') + '_' + df_tr['Start Hour'].astype('str')
for block in ['30', '15', '10']:
    df_tr[f'Unique {block}-Minute Block'] = df_tr[
        'Unique Hour'] + '_' + df_tr[
        f'Start {block}-Minute Block'].astype('str')


df_tr.head(5)

,Unix_Test_Start_Time,Local_Test_Start_Time,Unix_Test_End_Time,Local_Test_End_Time,Verse_ID,Verse_Code,Verse,Characters,WPM,Test_Seconds,Error_Rate,Error_and_Backspace_Rate,Marathon_Mode,Player,Tag_1,Tag_2,Tag_3,Test number,Start Date,Start Hour,Start Minute,Start 30-Minute Block,Start 15-Minute Block,Start 10-Minute Block,End Date,End Hour,End Minute,End 30-Minute Block,End 15-Minute Block,End 10-Minute Block,Unique Hour,Unique 30-Minute Block,Unique 15-Minute Block,Unique 10-Minute Block
0,1750909743,2025-06-25 23:49:03-04:00,1750909747,2025-06-25 23:49:07-04:00,1,Genesis_1:1,"In the beginning, God created heaven and earth.",47,124.928204,4.514593,0.021277,0.042553,0,KJB3,Cherry_Red,NaN,NaN,1,2025-06-25,23,49,2,4,5,2025-06-25,23,49,2,4,5,2025-06-25_23,2025-06-25_23_2,2025-06-25_23_4,2025-06-25_23_5
1,1750910427,2025-06-26 00:00:27-04:00,1750910430,2025-06-26 00:00:30-04:00,1,Genesis_1:1,"In the beginning, God created heaven and earth.",47,181.050428,3.115154,0.000000,0.000000,0,KJB3,Cherry_Red,NaN,NaN,2,2025-06-26,0,0,1,1,1,2025-06-26,0,0,1,1,1,2025-06-26_0,2025-06-26_0_1,2025-06-26_0_1,2025-06-26_0_1
2,1750910839,2025-06-26 00:07:19-04:00,1750910854,2025-06-26 00:07:34-04:00,2,Genesis_1:2,"But the earth was empty and unoccupied, and da...",141,113.465015,14.912085,0.120567,0.205674,1,KJB3,Cherry_Red,NaN,NaN,3,2025-06-26,0,7,1,1,1,2025-06-26,0,7,1,1,1,2025-06-26_0,2025-06-26_0_1,2025-06-26_0_1,2025-06-26_0_1
3,1750910854,2025-06-26 00:07:34-04:00,1750910860,2025-06-26 00:07:40-04:00,3,Genesis_1:3,"And God said, ""Let there be light."" And light ...",53,110.773001,5.741471,0.018868,0.037736,1,KJB3,Cherry_Red,NaN,NaN,4,2025-06-26,0,7,1,1,1,2025-06-26,0,7,1,1,1,2025-06-26_0,2025-06-26_0_1,2025-06-26_0_1,2025-06-26_0_1
4,1750910860,2025-06-26 00:07:40-04:00,1750910869,2025-06-26 00:07:49-04:00,4,Genesis_1:4,"And God saw the light, that it was good; and s...",89,116.464066,9.170210,0.078652,0.157303,1,KJB3,Cherry_Red,NaN,NaN,5,2025-06-26,0,7,1,1,1,2025-06-26,0,7,1,1,1,2025-06-26_0,2025-06-26_0_1,2025-06-26_0_1,2025-06-26_0_1


## Determining how many characters the player typed after each *rolling* hour, 30-minute period, 15-minute period, and 10-minute period:

(This information will be helpful for calculating endurance-based statistics.)

In [4]:
col_seconds_pair_list = [['Characters Typed in Next Hour', 3600],
                         ['Characters Typed in Next 30 Minutes', 1800],
                         ['Characters Typed in Next 15 Minutes', 900],
                         ['Characters Typed in Next 10 Minutes', 600]]

Note: the following code will likely take a considerable amount of time to run once users have completed a significant number of tests (e.g. 1000+). Therefore, it would be ideal to eventually replace it with C++-based code *or* a more sophisticated Python-based approach. (I tried out the 'numba' engine setting within the df.apply() function, but it failed to work correctly.)

In the meantime, I've commented out this code so that it won't cause performance issues going forward.

In [5]:
# df_tr_condensed = df_tr[['Unix_Test_Start_Time', 
# 'Unix_Test_End_Time', 'Characters', 'Test number']].copy()
# I thought this DataFrame might be more compatible with certain
# alternative engine options, but unfortunately, that wasn't the case.

In [6]:
# for col_seconds_pair in col_seconds_pair_list:

#     df_tr[col_seconds_pair[0]] = df_tr.apply(
#         lambda x: df_tr[(df_tr[
#             'Unix_Test_Start_Time'] >= x['Unix_Test_Start_Time']) & (
#             df_tr['Unix_Test_End_Time'] 
#         < (x['Unix_Test_Start_Time'] + col_seconds_pair[1]))][
#             'Characters'].sum(), axis = 1)
# df_tr

Note: I thought the following approach might actually be faster than the above option, as it only requires a single loop through the whole DataFrame. However, I found it to take a bit longer than the previous method.

In [7]:
# for col in ['characters_typed_in_next_hour',
#             'characters_typed_in_next_30_minutes',
#             'characters_typed_in_next_15_minutes',
#             'characters_typed_in_next_10_minutes']:
#     df_tr[col] = 0
# for i in range(len(df_tr)):
#     start_time = df_tr.iloc[i]['Unix_Test_Start_Time'].astype(
#         'int64')
    
#     df_tr.iloc[i, df_tr.columns.get_loc(
#         'characters_typed_in_next_hour')] = df_tr[(
#         df_tr[
#         'Unix_Test_Start_Time'] >= start_time) & (df_tr[
#             'Unix_Test_End_Time'] 
#         < (start_time + 3600))]['Characters'].sum()
    
#     df_tr.iloc[i, df_tr.columns.get_loc(
#         'characters_typed_in_next_30_minutes')] = df_tr[(
#         df_tr[
#         'Unix_Test_Start_Time'] >= start_time) & (df_tr[
#             'Unix_Test_End_Time'] 
#         < (start_time + 1800))]['Characters'].sum()

#     df_tr.iloc[i, df_tr.columns.get_loc(
#             'characters_typed_in_next_15_minutes')] = df_tr[(
#             df_tr[
#             'Unix_Test_Start_Time'] >= start_time) & (df_tr[
#                 'Unix_Test_End_Time'] 
#             < (start_time + 900))]['Characters'].sum()
    
#     df_tr.iloc[i, df_tr.columns.get_loc(
#             'characters_typed_in_next_10_minutes')] = df_tr[(
#             df_tr[
#             'Unix_Test_Start_Time'] >= start_time) & (df_tr[
#                 'Unix_Test_End_Time'] 
#             < (start_time + 600))]['Characters'].sum()
            

In [8]:
df_tr

,Unix_Test_Start_Time,Local_Test_Start_Time,Unix_Test_End_Time,Local_Test_End_Time,Verse_ID,Verse_Code,Verse,Characters,WPM,Test_Seconds,Error_Rate,Error_and_Backspace_Rate,Marathon_Mode,Player,Tag_1,Tag_2,Tag_3,Test number,Start Date,Start Hour,Start Minute,Start 30-Minute Block,Start 15-Minute Block,Start 10-Minute Block,End Date,End Hour,End Minute,End 30-Minute Block,End 15-Minute Block,End 10-Minute Block,Unique Hour,Unique 30-Minute Block,Unique 15-Minute Block,Unique 10-Minute Block
0,1750909743,2025-06-25 23:49:03-04:00,1750909747,2025-06-25 23:49:07-04:00,1,Genesis_1:1,"In the beginning, God created heaven and earth.",47,124.928204,4.514593,0.021277,0.042553,0,KJB3,Cherry_Red,NaN,NaN,1,2025-06-25,23,49,2,4,5,2025-06-25,23,49,2,4,5,2025-06-25_23,2025-06-25_23_2,2025-06-25_23_4,2025-06-25_23_5
1,1750910427,2025-06-26 00:00:27-04:00,1750910430,2025-06-26 00:00:30-04:00,1,Genesis_1:1,"In the beginning, God created heaven and earth.",47,181.050428,3.115154,0.000000,0.000000,0,KJB3,Cherry_Red,NaN,NaN,2,2025-06-26,0,0,1,1,1,2025-06-26,0,0,1,1,1,2025-06-26_0,2025-06-26_0_1,2025-06-26_0_1,2025-06-26_0_1
2,1750910839,2025-06-26 00:07:19-04:00,1750910854,2025-06-26 00:07:34-04:00,2,Genesis_1:2,"But the earth was empty and unoccupied, and da...",141,113.465015,14.912085,0.120567,0.205674,1,KJB3,Cherry_Red,NaN,NaN,3,2025-06-26,0,7,1,1,1,2025-06-26,0,7,1,1,1,2025-06-26_0,2025-06-26_0_1,2025-06-26_0_1,2025-06-26_0_1
3,1750910854,2025-06-26 00:07:34-04:00,1750910860,2025-06-26 00:07:40-04:00,3,Genesis_1:3,"And God said, ""Let there be light."" And light ...",53,110.773001,5.741471,0.018868,0.037736,1,KJB3,Cherry_Red,NaN,NaN,4,2025-06-26,0,7,1,1,1,2025-06-26,0,7,1,1,1,2025-06-26_0,2025-06-26_0_1,2025-06-26_0_1,2025-06-26_0_1
4,1750910860,2025-06-26 00:07:40-04:00,1750910869,2025-06-26 00:07:49-04:00,4,Genesis_1:4,"And God saw the light, that it was good; and s...",89,116.464066,9.170210,0.078652,0.157303,1,KJB3,Cherry_Red,NaN,NaN,5,2025-06-26,0,7,1,1,1,2025-06-26,0,7,1,1,1,2025-06-26_0,2025-06-26_0_1,2025-06-26_0_1,2025-06-26_0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1752552952,2025-07-15 00:15:52-04:00,1752552962,2025-07-15 00:16:02-04:00,83,Genesis_4:3,"Then it happened, after many days, that Cain o...",101,124.332020,9.748092,0.059406,0.079208,1,KJB3,Cherry_Red,NaN,NaN,97,2025-07-15,0,15,1,2,2,2025-07-15,0,16,1,2,2,2025-07-15_0,2025-07-15_0_1,2025-07-15_0_2,2025-07-15_0_2
97,1752552962,2025-07-15 00:16:02-04:00,1752552975,2025-07-15 00:16:15-04:00,84,Genesis_4:4,Abel likewise offered from the firstborn of hi...,128,120.063965,12.793181,0.054688,0.109375,1,KJB3,Cherry_Red,NaN,NaN,98,2025-07-15,0,16,1,2,2,2025-07-15,0,16,1,2,2,2025-07-15_0,2025-07-15_0_1,2025-07-15_0_2,2025-07-15_0_2
98,1752552984,2025-07-15 00:16:24-04:00,1752552996,2025-07-15 00:16:36-04:00,85,Genesis_4:5,"Yet in truth, he did not look with favor on Ca...",120,121.676182,11.834691,0.058333,0.116667,0,KJB3,Cherry_Red,NaN,NaN,99,2025-07-15,0,16,1,2,2,2025-07-15,0,16,1,2,2,2025-07-15_0,2025-07-15_0_1,2025-07-15_0_2,2025-07-15_0_2
99,1752553003,2025-07-15 00:16:43-04:00,1752553009,2025-07-15 00:16:49-04:00,86,Genesis_4:6,"And the Lord said to him: ""Why are you angry? ...",74,143.780078,6.176099,0.000000,0.000000,0,KJB3,Cherry_Red,NaN,NaN,100,2025-07-15,0,16,1,2,2,2025-07-15,0,16,1,2,2,2025-07-15_0,2025-07-15_0_1,2025-07-15_0_2,2025-07-15_0_2


### WPM results by test:

In [9]:
fig_wpm_by_test = px.line(
    df_tr, x = 'Test number', y = 'WPM',
title = 'WPM by Test Number',)
fig_wpm_by_test.write_html('Single_Player/WPM_by_race.html',
                           include_plotlyjs = 'cdn')

### Average WPM by Tag 1 values:

In [10]:
df_wpm_by_tag_1 = df_tr.pivot_table(index = 'Tag_1', values = 'WPM',
                                 aggfunc = 'mean').reset_index()
df_wpm_by_tag_1

,Tag_1,WPM
0,Cherry_Red,131.810269
1,Ga15pyd_Laptop,114.402321


In [11]:
fig_wpm_by_tag_1 = px.bar(df_wpm_by_tag_1, x = 'Tag_1', y = 'WPM',
                         title = 'Mean WPM by Tag_1 value')
fig_wpm_by_tag_1.write_html('Single_Player/Mean_WPM_by_Tag_1.html',
                            include_plotlyjs = 'cdn')

## Creating endurance-related charts:

## Visualizing *rolling* endurance statistics:

(I commented out the following visualization code because it relies on a very inefficient set of code that I have also commented out.)

In [12]:
# for col in [pair[0] for pair in col_seconds_pair_list]:

#     df_endurance = df_tr.sort_values(col, ascending = False).copy(
#     ).reset_index(drop=True).head(50)
#     df_endurance['Rank'] = (df_endurance.index + 1)

#     fig_endurance = px.bar(
#     df_endurance, x = 'Rank', 
#     y = col,
#     title = 'Most ' + col,
#     hover_data = ['Test number', 'Local_Test_Start_Time'])
    
#     fig_endurance.write_html('Single_Player/Endurance_Top_50_rolling_'+col.replace(
#         ' ', '_')+'.html', 
#     include_plotlyjs = 'cdn')

## Visualizing clock-based endurance statistics:

In [13]:
for time_category in ['Hour', '30-Minute Block', '15-Minute Block',
                      '10-Minute Block']:
    # The following code helps confirm that the following query() statement
    # is correctly filtering out tests whose starting and ending 
    # time categories don't match.
    # print(len(df_tr.query(f"`Start {time_category}` == `End {time_category}`")))
    df_endurance = df_tr.query(f"`Start {time_category}` == `End {time_category}`").pivot_table(
        index = f'Unique {time_category}', values = 'Characters', 
    aggfunc = 'sum').reset_index().sort_values(
    'Characters', ascending = False).reset_index(drop=True).head(50)
    df_endurance['Rank'] = (df_endurance.index + 1)
    
    fig_endurance = px.bar(
    df_endurance, x = f'Unique {time_category}', 
    y = 'Characters',
    title = 'Most Characters Typed By ' + time_category,
    hover_data = 'Rank')
    
    fig_endurance.write_html(
    'Single_Player/Endurance_Top_50_Clock_'+time_category.replace(
    ' ', '_')+'.html', 
    include_plotlyjs = 'cdn')

Graphing keypresses by date (in both chronological and ranked order):

In [14]:
df_top_dates_by_keypresses = df_tr.query(
    "`Start Date` == `End Date`").pivot_table(
    index = 'Start Date', values = 'Characters', 
    aggfunc = 'sum').reset_index().sort_values(
    'Characters', ascending = False).reset_index(drop=True)
df_top_dates_by_keypresses['Rank'] = df_top_dates_by_keypresses.index + 1
# Plotly will automatically arrange these dates in chronological order 
# despite our having sorted the source DataFrame by characters.
fig_keypresses_by_date = px.bar(df_top_dates_by_keypresses,
                                     x = 'Start Date', y = 'Characters',
                              title = 'Characters typed by date',
                               hover_data = ['Rank'])
fig_keypresses_by_date.write_html('Single_Player/Keypresses_Typed_by_Date.html',
                                  include_plotlyjs='cdn')

In [15]:
fig_top_dates_by_keypresses = px.bar(
    df_top_dates_by_keypresses.head(50),
       x = 'Start Date', y = 'Characters',
       title = 'Dates with the most characters typed', 
       hover_data = ['Rank']).update_layout(
    xaxis_type = 'category')
fig_top_dates_by_keypresses.write_html(
    'Single_Player/Top_Dates_by_Keypresses.html', include_plotlyjs='cdn')

In [16]:
end_time = time.time()
run_time = end_time - start_time
print(f"Finished calculating and visualizing single-player stats in \
{round(run_time, 3)} seconds.")

Finished calculating and visualizing single-player stats in 0.713 seconds.
